<a href="https://colab.research.google.com/github/DrZedd42/stylegan2/blob/master/Gary_Image_Minting_Printer_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
 
!git clone https://github.com/dvschultz/stylegan2
!pip install opensimplex # needed for noise interpolation
%cd stylegan2
%mkdir datasets

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 549, done.
remote: Total 549 (delta 0), reused 0 (delta 0), pack-reused 549
Receiving objects: 100% (549/549), 22.46 MiB | 30.95 MiB/s, done.
Resolving deltas: 100% (286/286), done.
/content/stylegan2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download the model of choice
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
 
 
import pretrained_networks
 
 
network_pkl= "/content/drive/My Drive/GAN/network-snapshot-008040.pkl"
 
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

Loading networks from "/content/drive/My Drive/GAN/network-snapshot-008040.pkl"...


#Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). I recommend doing this on your server because the files become quite large and will be slow to upload over FTP.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [ ]:
!python dataset_tool.py create_from_images ./datasets/my-custom-dataset ./my-custom-images

Loading images from "./my-custom-images"
Error: No input images found


#Training
I do not recommend attempting to train your model on Colab. It will be a very tedious process. Here are the steps, however.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.


Once running, your training files will show up in the results folder.

copied from stylegan exampke
connects google drive.

#Test the model
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
!python run_generator.py generate-images --network=/content/drive/My\ Drive/GAN/network-snapshot-008040.pkl --seeds=5949-6000 --truncation-psi=0.0001
#!python run_generator.py generate-images --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00005-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/drive/My Drive/GAN/network-snapshot-008040.pkl"...
Generating image for seed 5949 (1/52) ...
Generating image for seed 5950 (2/52) ...
Generating image for seed 5951 (3/52) ...
Generating image for seed 5952 (4/52) ...
Generating image for seed 5953 (5/52) ...
Generating image for seed 5954 (6/52) ...
Generating image for seed 5955 (7/52) ...
Generating image for seed 5956 (8/52) ...
Generating image for seed 5957 (9/52) ...
Generating image for seed 5958 (10/52) ...
Generating image for seed 5959 (11/52) ...
Generating image for seed 5960 (12/52) ...
Generating image for seed 5961 (13/52) ...
Generating image for seed 5962 (14/52) ...
Generating image for seed 5963 (15/52) ...
Generating image for seed 5964 (16/52) ...
Generating image for seed 5965 (17/52) ...
Generating image for seed 5966 (18/52) ...
Generating image for seed 5967 (19

Let’s zip the generated files and download them.

In [ ]:
 
!cd ..
!cd ..
#!ls
!dir
#/content/drive/My Drive/
!zip -r /content/drive/My\ Drive/Images/garystartPrint5.zip /content/stylegan2/results/00005-generate-images/*.png

datasets		results
dataset_tool.py		run_generator.py
dnnlib			run_metrics.py
Dockerfile		run_projector.py
docs			run_training.py
easy-install.sh		runway_model.py
LICENSE.txt		runway.yml
metrics			StyleGAN2_Google_Colab_Example.ipynb
pretrained_networks.py	StyleGAN2_projection_interpolation.ipynb
projector.py		StyleGAN2_Projection.ipynb
__pycache__		test_nvcc.cu
README.md		training
requirements.txt	USAGE.md
  adding: content/stylegan2/results/00005-generate-images/ (stored 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5988.png (deflated 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5982.png (deflated 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5974.png (deflated 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5976.png (deflated 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5952.png (deflated 0%)
  adding: content/stylegan2/results/00005-generate-images/seed5964.png (deflated 0%)
  adding: 

#Interpolation


In [ ]:
!python run_generator.py generate-latent-walk --network=/content/drive/My\ Drive/GAN/network-snapshot-008040.pkl --seeds=3,11,17,25,3,44,3333,222,666 --frames 400 --truncation-psi=0.7

Local submit - run_dir: results/00009-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/drive/My Drive/GAN/network-snapshot-008040.pkl"...
not enough values to generate walk
Generating image for step 0/408 ...
Generating image for step 1/408 ...
Generating image for step 2/408 ...
Generating image for step 3/408 ...
Generating image for step 4/408 ...
Generating image for step 5/408 ...
Generating image for step 6/408 ...
Generating image for step 7/408 ...
Generating image for step 8/408 ...
Generating image for step 9/408 ...
Generating image for step 10/408 ...
Generating image for step 11/408 ...
Generating image for step 12/408 ...
Generating image for step 13/408 ...
Generating image for step 14/408 ...
Generating image for step 15/408 ...
Generating image for step 16/408 ...
Generating image for step 17/408 ...
Generating image for step 18/408 ...
Generating image for step 19/408 ...
Generating image for ste

In [ ]:
#convert to video 
!ffmpeg -r 24 -i /content/stylegan2/results/00009-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
 
import drive
file = drive.CreateFile({'parents':[{u'id': images}]})
file.SetContentFile('my_image.png')
file.Upload()

NameError: ignored